# DDoS Classifier

# CICFlowMeter Data

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import *
import pickle
import time
from sklearn.utils.class_weight import compute_class_weight

In [2]:
import tensorflow as tf
import keras as kr

/home/talhajavaid32/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2

In [4]:
def create_new_experiment_dir(exp_base_dir):
    def get_unique_name(): return time.strftime("%Y%m%d-%H%M%S")

    exp_base_dir = os.path.abspath(os.path.expanduser(exp_base_dir))
    if exp_base_dir[-1] != '/':
        exp_base_dir += '/'

    # Create new experiment directory
    exp_dir = exp_base_dir + get_unique_name() + '/'
    os.makedirs(exp_dir)

    os.makedirs(exp_dir + 'pickle_dump/')
    os.makedirs(exp_dir + 'logs/')

    return exp_dir

### F1 Score

In [5]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

### Confusion Matrix

In [6]:
def print_confusion_matrix(plabels,tlabels):
    """
        functions print the confusion matrix for the different classes
        to find the error...
        
        Input:
        -----------
        plabels: predicted labels for the classes...
        tlabels: true labels for the classes
        
        code from: http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python
    """
    import pandas as pd
    plabels = pd.Series(np.squeeze(plabels))
    tlabels = pd.Series(np.squeeze(tlabels))
    
    # draw a cross tabulation...
    df_confusion = pd.crosstab(tlabels,plabels, rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    #print df_confusion
    return df_confusion

# Preprocessing Data

#### Load clean data

In [7]:
data_df = pd.read_csv('./data/IDS2017Data/DDoS-WednesdayData/CICFlowData_clean_merged.csv')
feat = list(data_df.columns)
feat.remove('flow_id')
feat.remove('label')

#### Ignoring encoding for now

#### One hot encode categorical

In [8]:
# categ_feat = ['source_port', 'destination_port', 'protocol']

# data_df = data.itervalues().next()
# data_df = pd.get_dummies(data_df, prefix=['sp', 'dp', 'pr'], columns=categ, sparse=True)
# data_df

In [9]:
# enc = OneHotEncoder()
# enc.fit(data_df)
# transformed = enc.transform(data_df)

#### Scaling data

#### Dropping features having very close max and min values to avoid NaNs on scaling

In [10]:
diff = data_df[feat].max(axis=0) - data_df[feat].min(axis=0)
eps = 0.00000000001

feat = list(data_df[feat].columns[~(diff.abs() < eps)])

## Dropping Categorical features Source port , Destination port and protocol for now

In [11]:
feat = filter(lambda x: x not in ['source_port', 'destination_port', 'protocol'], feat)

In [12]:
# Scaling only Numerical features
num_feat = feat

# Using MinMaxScaler
min_max = MinMaxScaler()
# Fit scaler
min_max.fit(data_df[num_feat].as_matrix())
# Transform data
data_df[num_feat] = pd.DataFrame(min_max.transform(
    data_df[num_feat].as_matrix()), columns=num_feat)

## Converting multiclass problem to binary classification

In [13]:
# data_df['label'][data_df['label'] != 'BENIGN'] = 'ATTACK'

## Labels

In [14]:
data_df.label.unique()

array(['BENIGN', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk',
       'DoS GoldenEye'], dtype=object)

In [15]:
# le = LabelEncoder()
# le.fit(data_df['label'].as_matrix())
# data_df['label'] = pd.DataFrame(le.transform(
#     data_df['label'].as_matrix()), columns=['label'])
labels = {'BENIGN': 0, 'DoS slowloris': 1, 'DoS Slowhttptest': 2, 'DoS Hulk': 3,
       'DoS GoldenEye': 4}
classes = len(labels.keys())

# data_df['enclabel'] = 0
# data_df.loc[data_df.label == 'ATTACK', 'enclabel'] = 1

for k, v in labels.iteritems():
    data_df.loc[data_df.label == k, 'enclabel'] = v

In [16]:
print 'Data Label counts:'
class_weights = {}
for k, v in labels.iteritems():
    count = (data_df.enclabel == v).sum()
    p = (count / float(data_df.shape[0])) * 100
    class_weights[v] = p
    print '%s flows count: %d  Percentage: %.2f%%' % (k, count, p)
comp_cls_wt = compute_class_weight(
    'balanced', data_df.enclabel.unique(), data_df.enclabel)
display(comp_cls_wt)

Data Label counts:
BENIGN flows count: 439683  Percentage: 46.44%
DoS GoldenEye flows count: 10293  Percentage: 1.09%
DoS Hulk flows count: 478648  Percentage: 50.56%
DoS Slowhttptest flows count: 5499  Percentage: 0.58%
DoS slowloris flows count: 12583  Percentage: 1.33%


array([ 0.43063116, 15.04738139, 34.43193308,  0.39557504, 18.39514233])

#### Splitting up data

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = data_df[feat].as_matrix()
# y = data_df['label'].as_matrix()
y = data_df['enclabel'].as_matrix()

In [19]:
# Splitting Train Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, stratify=y)

/home/talhajavaid32/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [20]:
# Splitting Train Validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.8, stratify=y_train)

In [21]:
print 'Train data:',(X_train.shape)
print 'Train labels:', (y_train.shape)
print 'Validation data:', (X_val.shape)
print 'Validation labels:', (y_val.shape)
print 'Test data:', (X_test.shape)
print 'Test labels', (y_test.shape)

Train data: (605891, 66)
Train labels: (605891,)
Validation data: (151473, 66)
Validation labels: (151473,)
Test data: (189342, 66)
Test labels (189342,)


In [22]:
print 'Training data:'
for k, v in labels.iteritems():
    print k, 'flows count:', (y_train == v).sum()
print '\n'

print 'Validation data:'
for k, v in labels.iteritems():
    print k, 'flows count:', (y_val == v).sum()
print '\n'

print 'Testing data:'
for k, v in labels.iteritems():
    print k, 'flows count:', (y_test == v).sum()
print '\n'

Training data:
BENIGN flows count: 281397
DoS GoldenEye flows count: 6588
DoS Hulk flows count: 306334
DoS Slowhttptest flows count: 3519
DoS slowloris flows count: 8053


Validation data:
BENIGN flows count: 70349
DoS GoldenEye flows count: 1647
DoS Hulk flows count: 76584
DoS Slowhttptest flows count: 880
DoS slowloris flows count: 2013


Testing data:
BENIGN flows count: 87937
DoS GoldenEye flows count: 2058
DoS Hulk flows count: 95730
DoS Slowhttptest flows count: 1100
DoS slowloris flows count: 2517




### Attack vs Benign

In [23]:
print "Benign flows: %.2f%%" % (
    ((data_df['enclabel'] == labels['BENIGN']).sum() / float(data_df.shape[0])) * 100)
print "Attack flows: %.2f%%" % (
    ((data_df['enclabel'] != labels['BENIGN']).sum() / float(data_df.shape[0])) * 100)

Benign flows: 46.44%
Attack flows: 53.56%


### One hot encode labels to use with softmax

In [24]:
# label_one_hot_enc = OneHotEncoder(n_values=classes)
# y_train_enc = label_one_hot_enc.fit_transform(y_train.reshape(-1, 1)).toarray()
# y_val_enc = label_one_hot_enc.transform(y_val.reshape(-1, 1)).toarray()
# y_test_enc = label_one_hot_enc.transform(y_test.reshape(-1, 1)).toarray()

# y_train_enc = np.zeros((y_train.shape[0], 2))
# y_train_enc[~y_train.astype('bool'), 0] = 1
# y_train_enc[y_train.astype('bool'), 1] = 1

# y_val_enc = np.zeros((y_val.shape[0], 2))
# y_val_enc[~y_val.astype('bool'), 0] = 1
# y_val_enc[y_val.astype('bool'), 1] = 1

# y_test_enc = np.zeros((y_test.shape[0], 2))
# y_test_enc[~y_test.astype('bool'), 0] = 1
# y_test_enc[y_test.astype('bool'), 1] = 1


###########################
y_train_enc = np.zeros((y_train.shape[0], classes))
for i in xrange(classes):
    y_train_enc[y_train == i, i] = 1

y_val_enc = np.zeros((y_val.shape[0], classes))
for i in xrange(classes):
    y_val_enc[y_val == i, i] = 1
    
y_test_enc = np.zeros((y_test.shape[0], classes))
for i in xrange(classes):
    y_test_enc[y_test == i, i] = 1

# Building the Model

In [25]:
# Training parameters
train_params = {}

train_params['batch_size'] = 1024 # 64 128 256 1024
train_params['epochs'] = 300
train_params['feat'] = X_train.shape[1]
train_params['classes'] = classes
train_params['alpha'] = 0.001
train_params['decay'] = 0.00005
train_params['loss'] = 'categorical_crossentropy'
train_params['optimizer'] = 'Adam'
train_params['reg'] = 0 # 0.001
train_params['dropout'] = 0.2 # Dropout rate

In [26]:
# Keras sequential model
model = Sequential()

model.add(Dense(128, input_shape=(
    train_params['feat'], ), kernel_initializer='he_normal',
    kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(256, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(128, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(64, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(32, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(16, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(8, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(train_params['classes'], kernel_initializer='VarianceScaling'))
model.add(BatchNormalization())
model.add(Activation('softmax'))

adam = Adam(lr=train_params['alpha'], decay=train_params['decay'])
model.compile(loss=train_params['loss'], optimizer=adam, metrics=[f1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               8576      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
__________

# New experiment

In [27]:
exp_base_dir = './experiments'
exp_dir = create_new_experiment_dir(exp_base_dir)
print 'New experiment directory:', exp_dir
pickle_dir = exp_dir + 'pickle_dump/'
log_dir = exp_dir + 'logs/'

print "Enter experiment summary. Enter empty line or Ctrl-D to save it."
exp_summ = []
while True:
    try:
        line = raw_input("")
    except EOFError:
        break
    if line == '':
        break
    exp_summ.append(line)
exp_summ = '\n'.join(exp_summ)
exp_summ += '\n'

# Save min_max_scaler for used for numerical features
with open(pickle_dir + 'min_max_data.p', 'wb') as f:
    pickle.dump(min_max, f)
# Save label_encoder
# with open(pickle_dir + 'label_encoder.p', 'wb') as f:
#     pickle.dump(le, f)
# Save label_one_hot_encoder
# with open(pickle_dir + 'label_one_hot_encoder.p', 'wb') as f:
#     pickle.dump(label_one_hot_enc, f)
# Save tuple of form (final data_df's feature's list, labels)
with open(pickle_dir + 'cols_features_labels.p', 'wb') as f:
    pickle.dump((list(data_df.columns), feat, labels), f)
# Save summaries
with open(exp_dir + 'model_summary', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
with open(exp_dir + 'experiment_summary', 'w') as f:
    f.write(exp_summ)
# Save Training parameters
with open(exp_dir + 'train_params', 'w') as f:
    for k, v in train_params.iteritems():
        f.write(k + ': ' + str(v) + '\n')

New experiment directory: /home/talhajavaid32/DDoSFence/ddosfenceai/experiments/20180422-083125/
Enter experiment summary. Enter empty line or Ctrl-D to save it.
Multiclass classifier with sklearn class weights to balance dataset
Added simulator data
learning rate changed to 0.001
epochs changed to 300
dropout changed to 0.2



# Training

In [28]:
# Training checkpoints
checkpoint = ModelCheckpoint(
    filepath=(exp_dir + 'model.h5'), verbose=1, save_best_only=True)
# Tensorboard logs
tensorboard = TensorBoard(log_dir=log_dir, batch_size=train_params['batch_size'])
# Train
model.fit(X_train, y_train_enc, batch_size=train_params['batch_size'], epochs=train_params['epochs'],
          validation_data=(X_val, y_val_enc), class_weight=comp_cls_wt, callbacks=[checkpoint, tensorboard])

Train on 605891 samples, validate on 151473 samples
Epoch 1/300
605891/605891 [==============================] - 18s 30us/step - loss: 0.7527 - f1: 0.6830 - val_loss: 0.3915 - val_f1: 0.9311
Epoch 2/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.3435 - f1: 0.9245 - val_loss: 0.2414 - val_f1: 0.9402
Epoch 3/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.2481 - f1: 0.9316 - val_loss: 0.1866 - val_f1: 0.9410
Epoch 4/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.2054 - f1: 0.9363 - val_loss: 0.1489 - val_f1: 0.9499
Epoch 5/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.1785 - f1: 0.9412 - val_loss: 0.1310 - val_f1: 0.9512
Epoch 6/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.1632 - f1: 0.9436 - val_loss: 0.1194 - val_f1: 0.9543
Epoch 7/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.1507 - f1: 0.9463 - val_loss:

Epoch 24/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.1036 - f1: 0.9593 - val_loss: 0.0770 - val_f1: 0.9661
Epoch 25/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.1036 - f1: 0.9594 - val_loss: 0.0758 - val_f1: 0.9667
Epoch 26/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.1018 - f1: 0.9599 - val_loss: 0.0774 - val_f1: 0.9654
Epoch 27/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.1016 - f1: 0.9598 - val_loss: 0.0760 - val_f1: 0.9656
Epoch 28/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.1015 - f1: 0.9597 - val_loss: 0.0744 - val_f1: 0.9667
Epoch 29/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0998 - f1: 0.9606 - val_loss: 0.0752 - val_f1: 0.9674
Epoch 30/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.1003 - f1: 0.9603 - val_loss: 0.0742 - val_f1: 0.9674
Epoch 31/300
605891/

Epoch 50/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.0904 - f1: 0.9628 - val_loss: 0.0686 - val_f1: 0.9685
Epoch 51/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0902 - f1: 0.9628 - val_loss: 0.0681 - val_f1: 0.9688
Epoch 52/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0906 - f1: 0.9629 - val_loss: 0.0690 - val_f1: 0.9681
Epoch 53/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.0908 - f1: 0.9624 - val_loss: 0.0693 - val_f1: 0.9678
Epoch 54/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0895 - f1: 0.9630 - val_loss: 0.0668 - val_f1: 0.9696
Epoch 55/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0904 - f1: 0.9625 - val_loss: 0.0681 - val_f1: 0.9687
Epoch 56/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0897 - f1: 0.9630 - val_loss: 0.0679 - val_f1: 0.9689
Epoch 57/300
605891/

Epoch 78/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.0849 - f1: 0.9669 - val_loss: 0.0635 - val_f1: 0.9723
Epoch 79/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.0837 - f1: 0.9674 - val_loss: 0.0636 - val_f1: 0.9725
Epoch 80/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0838 - f1: 0.9674 - val_loss: 0.0644 - val_f1: 0.9718
Epoch 81/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0846 - f1: 0.9670 - val_loss: 0.0626 - val_f1: 0.9728
Epoch 82/300
605891/605891 [==============================] - 14s 23us/step - loss: 0.0839 - f1: 0.9671 - val_loss: 0.0639 - val_f1: 0.9718
Epoch 83/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0832 - f1: 0.9675 - val_loss: 0.0635 - val_f1: 0.9717
Epoch 84/300
605891/605891 [==============================] - 15s 24us/step - loss: 0.0837 - f1: 0.9676 - val_loss: 0.0623 - val_f1: 0.9728
Epoch 85/300
605891/

Epoch 106/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0807 - f1: 0.9689 - val_loss: 0.0608 - val_f1: 0.9738
Epoch 107/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0812 - f1: 0.9692 - val_loss: 0.0606 - val_f1: 0.9754
Epoch 108/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0803 - f1: 0.9691 - val_loss: 0.0600 - val_f1: 0.9739
Epoch 109/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0806 - f1: 0.9692 - val_loss: 0.0595 - val_f1: 0.9741
Epoch 110/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0805 - f1: 0.9693 - val_loss: 0.0579 - val_f1: 0.9768
Epoch 111/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0799 - f1: 0.9695 - val_loss: 0.0569 - val_f1: 0.9774
Epoch 112/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0794 - f1: 0.9694 - val_loss: 0.0583 - val_f1: 0.9765
Epoch 113/300

Epoch 136/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0774 - f1: 0.9706 - val_loss: 0.0576 - val_f1: 0.9766
Epoch 137/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0767 - f1: 0.9707 - val_loss: 0.0569 - val_f1: 0.9768
Epoch 138/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0768 - f1: 0.9705 - val_loss: 0.0589 - val_f1: 0.9748
Epoch 139/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0771 - f1: 0.9707 - val_loss: 0.0535 - val_f1: 0.9797
Epoch 140/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0764 - f1: 0.9708 - val_loss: 0.0553 - val_f1: 0.9782
Epoch 141/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0769 - f1: 0.9709 - val_loss: 0.0546 - val_f1: 0.9781
Epoch 142/300
605891/605891 [==============================] - 14s 24us/step - loss: 0.0771 - f1: 0.9706 - val_loss: 0.0567 - val_f1: 0.9774
Epoch 143/300

KeyboardInterrupt: 

### Experiment notes

In [29]:
print "Enter experiment summary. Enter empty line or Ctrl-D to save it."
exp_summ = []
while True:
    try:
        line = raw_input("")
    except EOFError:
        break
    if line == '':
        break
    exp_summ.append(line)
exp_summ = '\n'.join(exp_summ)
exp_summ += '\n'

with open(exp_dir + 'experiment_summary', 'a') as f:
    f.write(exp_summ)

Enter experiment summary. Enter empty line or Ctrl-D to save it.
loss: 0.0771 - f1: 0.9707 - val_loss: 0.0535 - val_f1: 0.9797



# Testing

In [30]:
scores = model.evaluate(X_test, y_test_enc)
with open(exp_dir + 'test_results', 'w') as f:
    for i in xrange(len(scores)):
        s = "%s: %.2f" % (model.metrics_names[i], scores[i])
        print s
        f.write(s + '\n')

189342/189342 [==============================] - 30s 157us/step
loss: 0.06
f1: 0.98


## Test Set Confusion Matrix

In [31]:
y_pred = model.predict(X_test, batch_size=X_test.shape[0], verbose=1)
# y_pred = y_pred.nonzero()[1]

189342/189342 [==============================] - 0s 2us/step


In [32]:
# benign = y_pred[:, 0]
# attack = y_pred[:, 1]
# mask = np.zeros((y_pred.shape[0],), dtype='bool')
# mask[np.where(attack >= benign)[0]] = True

# predlabels = np.empty((y_pred.shape[0],), dtype='object')
# predlabels[mask] = 'ATTACK'
# predlabels[~mask] = 'BENIGN'

# y_testlabels = np.empty((y_test.shape[0],), dtype='object')
# y_testlabels[y_test == 1] = 'ATTACK'
# y_testlabels[y_test == 0] = 'BENIGN'



# Flip labels dict
labels_inv = {v: k for k, v in labels.iteritems()}

predlabels = np.empty((y_pred.shape[0],), dtype='object')
maxidx = np.argmax(y_pred, axis=1)
for i in xrange(classes):
    predlabels[maxidx == i] = labels_inv[i]

y_testlabels = np.empty((y_test.shape[0],), dtype='object')
for i in xrange(classes):
    y_testlabels[y_test == i] = labels_inv[i]

In [33]:
confusion_matrix = print_confusion_matrix(predlabels, y_testlabels)
confusion_matrix.to_csv(exp_dir + 'confusion_matrix.csv')
confusion_matrix

Predicted         BENIGN  DoS GoldenEye  DoS Hulk  DoS Slowhttptest  \
Actual                                                                
BENIGN             84167              7      3723                32   
DoS GoldenEye          3           1882       172                 1   
DoS Hulk              96             90     95537                 4   
DoS Slowhttptest       6              1         4              1085   
DoS slowloris          4              1         3                 8   
All                84276           1981     99439              1130   

Predicted         DoS slowloris     All  
Actual                                   
BENIGN                        8   87937  
DoS GoldenEye                 0    2058  
DoS Hulk                      3   95730  
DoS Slowhttptest              4    1100  
DoS slowloris              2501    2517  
All                        2516  189342

## Test on saved model

In [7]:
expdir = './experiments/20180418-221808/'
model = load_model(expdir + 'model.h5', {'f1': f1})

In [26]:
scores = model.evaluate(X_test, y_test_enc)
with open(expdir + 'test_results', 'w') as f:
    for i in xrange(len(scores)):
        s = "%s: %.2f" % (model.metrics_names[i], scores[i])
        print s
        f.write(s + '\n')

138279/138279 [==============================] - 19s 140us/step
loss: 0.09
f1: 0.97


In [27]:
y_pred = model.predict(X_test, batch_size=X_test.shape[0], verbose=1)

138279/138279 [==============================] - 0s 2us/step


In [28]:
# Flip labels dict
labels_inv = {v: k for k, v in labels.iteritems()}

predlabels = np.empty((y_pred.shape[0],), dtype='object')
maxidx = np.argmax(y_pred, axis=1)
for i in xrange(classes):
    predlabels[maxidx == i] = labels_inv[i]

y_testlabels = np.empty((y_test.shape[0],), dtype='object')
for i in xrange(classes):
    y_testlabels[y_test == i] = labels_inv[i]

In [30]:
confusion_matrix = print_confusion_matrix(predlabels, y_testlabels)
confusion_matrix.to_csv(expdir + 'confusion_matrix.csv')
confusion_matrix

Predicted         BENIGN  DoS GoldenEye  DoS Hulk  DoS slowloris     All
Actual                                                                  
BENIGN             86082             41      1813              0   87936
DoS GoldenEye         33           2021         3              2    2059
DoS Hulk             422              0     45603              0   46025
DoS Slowhttptest      55           1045         0              0    1100
DoS slowloris        158            999         2              0    1159
All                86750           4106     47421              2  138279

In [14]:
from keras.utils import plot_model
plot_model(model, to_file=expdir + 'model.png', rankdir='LR')